###### 2020-11-24 화요일

# 01_텍스트데이터토큰화 & 토큰사전저장

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import re
import json

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 1. 데이터 불러오기

In [2]:
d1             = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data1.csv')
d2             = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data2.csv')
d3             = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data3.csv', encoding='cp949')
d4             = pd.read_excel('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data4.xlsx')
d5             = pd.read_excel('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data5.csv')
d6             = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/division_data6.csv')
exist_label_df = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/exist_label_df.csv')
add_comment_df = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/add_comment.csv', encoding='cp949')

In [3]:
add_comment_df = add_comment_df.reset_index()
add_comment_df.columns = ['Unnamed: 0', '댓글', '악플여부']

In [4]:
# 모든데이터 하나의 데이터프레임으로 합치기
df_list = [d1, d2, d3, d4, d5, d6, exist_label_df, add_comment_df]

all_df = d1
for df in df_list[1:]:
    all_df = pd.concat([all_df, df])

In [5]:
# 필요없는 열 제거
all_df.drop(['Unnamed: 0', '출처'], axis=1, inplace=True)

# 라벨이 NaN인거 제거
index = all_df['악플여부'].isna()
raw_df = all_df[~index]

# 인덱스 리셋
raw_df = raw_df.reset_index()
raw_df.drop(['index'], inplace=True, axis=1)

In [6]:
# 원본유지 복사본 사용
copy_df = raw_df.copy()

In [7]:
# 고정 시드값 지정
seed = 123

# 댓글 길이 지정
comment_len = 400

##### 악플여부 라벨이 1인경우와 2인경우를 하나로 결합
 - 라벨이 1인 경우 제거하고 모델을 구현했을때, 악플여부가 애매한 댓글을 잘 구별하지 못함
 - 그래서 1인경우(악플 여부가 애매한 경우)와 2인경우(악플 여부가 명확한 경우)를 하나의 라벨로 만들어 모델로 구현

In [ ]:
index = copy_df['악플여부'] == 2
copy_df.loc[index, '악플여부'] = 1

In [ ]:
copy_df['악플여부'] = copy_df['악플여부'].apply(int)
copy_df['댓글'] = copy_df['댓글'].apply(str)

In [ ]:
copy_df['악플여부'].value_counts()

0    67435
1    22505
Name: 악플여부, dtype: int64

## 2. 토큰화 & 저장
 - feature와 target을 나눈다
 - feature를 음절을 기준으로 나눈다
 - feature와 target, 음절토큰사전을 저장한다
    - 이렇게 저장하는 이유는? : 매번 음절토큰사전을 만드는데 시간이 많이 소요되기 때문에 한번 만들어놓고 재사용하여 시간을 절약!


In [ ]:
def tokenization_save(df):

    feature = df['댓글']
    label = df['악플여부']

    feature_split = feature.apply(list).tolist()
    feature_token_list = sum(feature_split, [])
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(feature_token_list)
    
    word_index_vocab = tokenizer.word_index

    return feature, label, word_index_vocab


In [ ]:
def save_feature_target(feature, target) :
    save_df_list = [feature, target]
    file_names = ['feature', 'target']

    for idx, set in enumerate(save_df_list):
        set.to_csv('/content/drive/MyDrive/[final_project]_악플원정대/03_K-Fold_O_[label3개]/data/' + file_names[idx] + '.csv', encoding='UTF-8-SIG')


def save_word_index(word_index_vocab) :
    json_file = json.dumps(word_index_vocab, ensure_ascii=False)
    f = open('/content/drive/MyDrive/[final_project]_악플원정대/03_K-Fold_O_[label3개]/data/word_index_vocab.json', 'w')
    f.write(json_file)
    f.close()

In [ ]:
feature, target, word_index_vocab = tokenization_save(copy_df)

In [ ]:
# ' '(공백)도 사전에 포함 
word_index_vocab[' '] = len(word_index_vocab) + 1

In [ ]:
save_feature_target(feature, target)
save_word_index(word_index_vocab)